In [1]:
# ==============================================================================
# SEKCJA 1: SETUP I WCZYTANIE DANYCH
# ==============================================================================
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras import layers
import os
import json
import joblib
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

print("TensorFlow version:", tf.__version__)

# Wczytujemy przetworzone dane
print("Wczytuję przetworzone dane z pliku 'df_flats_processed.pkl'...")
df_flats = pd.read_pickle('df_flats_processed.pkl')
print(f"Dane wczytane. Liczba ogłoszeń: {len(df_flats)}")

# Definiujemy katalog z artefaktami
ARTIFACTS_DIR = '1_Artifacts_Polska'

TensorFlow version: 2.19.0
Wczytuję przetworzone dane z pliku 'df_flats_processed.pkl'...
Dane wczytane. Liczba ogłoszeń: 1100194


In [3]:
# ==============================================================================
# SEKCJA 2: OCENA SKUTECZNOŚCI MODELI (POPRAWIONA)
# ==============================================================================

# Definicje cech
numerical_features = ['Area', 'Price', 'NumberOfRooms', 'Floor', 'Floors', 'BuiltYear']
categorical_features = ['BuldingType']
text_based_features = ['text_district_id', 'text_subdistrict_id', 'text_street_id']
similarity_based_features = ['similar_district_id', 'similar_subdistrict_id', 'similar_street_id']

# Słownik do przechowywania wyników
evaluation_results = {}

# Pętla po miastach, dla których istnieją modele
for city_name in ['Łódź', 'Kraków', 'Warszawa', 'Poznań', 'Wrocław']:
    model_path = os.path.join(ARTIFACTS_DIR, f'{city_name}_location_model.keras')
    if not os.path.exists(model_path):
        print(f"Brak modelu dla {city_name}, pomijam.")
        continue

    print(f"\n--- Ocena modelu dla miasta: {city_name} ---")
    
    # Krok 1: Przygotowanie danych
    df_city = df_flats[df_flats['city_name'] == city_name]
    full_data = df_city[(df_city['loc_5'] != 0) & (df_city['loc_6'] != 0) & (df_city['loc_7'] != 0)].copy()

    if len(full_data) < 100:
        print(f"Zbyt mało danych dla {city_name} do przeprowadzenia rzetelnej oceny.")
        continue

    # Krok 2: Podział na zbiór treningowy i testowy (80/20)
    try:
        train_df, test_df = train_test_split(full_data, test_size=0.2, random_state=42, stratify=full_data['loc_5'])
    except ValueError:
        train_df, test_df = train_test_split(full_data, test_size=0.2, random_state=42)
    
    print(f"Rozmiar zbioru treningowego: {len(train_df)}, Rozmiar zbioru testowego: {len(test_df)}")

    # Krok 3: Przygotowanie cech (X) i etykiet (y)
    # --- Zbiór treningowy ---
    X_train_num = train_df[numerical_features].values
    ohe_columns = pd.get_dummies(train_df[categorical_features], prefix='type').columns.tolist()
    X_train_cat_dummies = pd.get_dummies(train_df[categorical_features], prefix='type').reindex(columns=ohe_columns, fill_value=0).values
    X_train_text = train_df[text_based_features].values
    X_train_sim = train_df[similarity_based_features].values
    
    scaler = MinMaxScaler()
    X_train_num_scaled = scaler.fit_transform(X_train_num)
    X_train = np.hstack([X_train_num_scaled, X_train_cat_dummies, X_train_text, X_train_sim])

    # --- Enkodery i etykiety treningowe ---
    encoders = {}
    targets_train = {}
    target_cols = {'district': 'loc_5', 'subdistrict': 'loc_6', 'street': 'loc_7'}
    
    for name, col in target_cols.items():
        le = LabelEncoder()
        targets_train[name] = le.fit_transform(train_df[col])
        encoders[name] = le # Zapisujemy cały obiekt enkodera
    
    # Krok 4: Definicja i trening nowego modelu
    input_shape = (X_train.shape[1],)
    input_layer = layers.Input(shape=input_shape)
    x = layers.Dense(128, activation='relu')(input_layer)
    x = layers.Dropout(0.3)(x)
    x = layers.Dense(64, activation='relu')(x)
    
    output_district = layers.Dense(len(encoders['district'].classes_), activation='softmax', name='district_output')(x)
    output_subdistrict = layers.Dense(len(encoders['subdistrict'].classes_), activation='softmax', name='subdistrict_output')(x)
    output_street = layers.Dense(len(encoders['street'].classes_), activation='softmax', name='street_output')(x)
    
    model = keras.Model(inputs=input_layer, outputs=[output_district, output_subdistrict, output_street])
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')
    
    print("Trenuję model na zbiorze treningowym...")
    model.fit(X_train, list(targets_train.values()), epochs=30, batch_size=32, verbose=0)

    # Krok 5: Przygotowanie zbioru testowego (tylko znane klasy)
    print("Przygotowuję zbiór testowy i dokonuję predykcji...")
    
    # *** KLUCZOWA ZMIANA TUTAJ ***
    # Filtrujemy zbiór testowy, aby zawierał tylko te etykiety, które model zna
    known_districts = encoders['district'].classes_
    known_subdistricts = encoders['subdistrict'].classes_
    known_streets = encoders['street'].classes_
    
    test_df_filtered = test_df[
        test_df['loc_5'].isin(known_districts) &
        test_df['loc_6'].isin(known_subdistricts) &
        test_df['loc_7'].isin(known_streets)
    ].copy()

    # Przygotowujemy cechy X dla odfiltrowanego zbioru testowego
    X_test_num = test_df_filtered[numerical_features].values
    X_test_cat_dummies = pd.get_dummies(test_df_filtered[categorical_features], prefix='type').reindex(columns=ohe_columns, fill_value=0).values
    X_test_text = test_df_filtered[text_based_features].values
    X_test_sim = test_df_filtered[similarity_based_features].values
    
    X_test_num_scaled = scaler.transform(X_test_num)
    X_test = np.hstack([X_test_num_scaled, X_test_cat_dummies, X_test_text, X_test_sim])

    # Przygotowujemy prawdziwe etykiety Y dla odfiltrowanego zbioru
    y_test_true_district = encoders['district'].transform(test_df_filtered['loc_5'])
    y_test_true_subdistrict = encoders['subdistrict'].transform(test_df_filtered['loc_6'])
    y_test_true_street = encoders['street'].transform(test_df_filtered['loc_7'])

    # Predykcja
    predictions = model.predict(X_test, verbose=0)
    
    # Krok 6: Obliczenie skuteczności
    pred_district_idx = np.argmax(predictions[0], axis=1)
    pred_subdistrict_idx = np.argmax(predictions[1], axis=1)
    pred_street_idx = np.argmax(predictions[2], axis=1)

    accuracy_district = accuracy_score(y_test_true_district, pred_district_idx)
    accuracy_subdistrict = accuracy_score(y_test_true_subdistrict, pred_subdistrict_idx)
    accuracy_street = accuracy_score(y_test_true_street, pred_street_idx)
    
    evaluation_results[city_name] = {
        'district_accuracy': accuracy_district,
        'subdistrict_accuracy': accuracy_subdistrict,
        'street_accuracy': accuracy_street
    }

    print(f"Skuteczność (Accuracy) dla {city_name} (na {len(test_df_filtered)} próbkach testowych):")
    print(f"  - Przewidywanie DZIELNICY: {accuracy_district:.2%}")
    print(f"  - Przewidywanie PODDZIELNICY: {accuracy_subdistrict:.2%}")
    print(f"  - Przewidywanie ULICY: {accuracy_street:.2%}")


--- Ocena modelu dla miasta: Łódź ---
Rozmiar zbioru treningowego: 14518, Rozmiar zbioru testowego: 3630
Trenuję model na zbiorze treningowym...
Przygotowuję zbiór testowy i dokonuję predykcji...
Skuteczność (Accuracy) dla Łódź (na 3614 próbkach testowych):
  - Przewidywanie DZIELNICY: 37.66%
  - Przewidywanie PODDZIELNICY: 22.66%
  - Przewidywanie ULICY: 9.82%

--- Ocena modelu dla miasta: Kraków ---
Rozmiar zbioru treningowego: 29057, Rozmiar zbioru testowego: 7265
Trenuję model na zbiorze treningowym...
Przygotowuję zbiór testowy i dokonuję predykcji...
Skuteczność (Accuracy) dla Kraków (na 7237 próbkach testowych):
  - Przewidywanie DZIELNICY: 39.57%
  - Przewidywanie PODDZIELNICY: 12.53%
  - Przewidywanie ULICY: 5.15%

--- Ocena modelu dla miasta: Warszawa ---
Rozmiar zbioru treningowego: 41968, Rozmiar zbioru testowego: 10493
Trenuję model na zbiorze treningowym...
Przygotowuję zbiór testowy i dokonuję predykcji...
Skuteczność (Accuracy) dla Warszawa (na 10460 próbkach testowych

In [4]:
# ==============================================================================
# SEKCJA 3: PODSUMOWANIE EFEKTYWNOŚCI
# ==============================================================================

print("\n--- Zbiorcze wyniki skuteczności modeli ---\n")
results_df = pd.DataFrame(evaluation_results).T # Transpozycja dla lepszej czytelności
results_df = results_df * 100 # Zamiana na procenty
print(results_df.to_string(float_format="%.2f%%"))


--- Zbiorcze wyniki skuteczności modeli ---

          district_accuracy  subdistrict_accuracy  street_accuracy
Łódź                 37.66%                22.66%            9.82%
Kraków               39.57%                12.53%            5.15%
Warszawa             23.23%                 9.50%            3.81%
Poznań               32.87%                15.92%            6.80%
Wrocław              40.94%                21.20%            5.03%
